In [ ]:
#in questo quaderno verranno valutati i modelli migliori su alcuni dataset di benchmark

In [ ]:
#il primo dataset scelto è SemEval_2017 task 4, in quanto è composto da tweet, ossia frasi brevi etichettate come pos/neu/neg
#gli altri file hanno reviews troppo lunghe e etichettate pos(neg e basta)

In [ ]:
#importa sem eveal test
#sposta cartelle modelli vincitori qui
#crea funzione univoca per valutazione
#scrivi email con risultati ai prof

In [44]:
import pandas as pd
sem_eval_test=pd.read_csv("C:\\Users\\Fossati\\Desktop\\Tesi\\Dati\\Dati_FineTuningBERT\\Train_test_polarity\\Evaluation_data\\Sem_Eval_EN\\sem_eval_test.csv")
sem_eval_test


,Unnamed: 0,Sentence,Polarity_Classification
0,0,#ArianaGrande Ari By Ariana Grande 80% Full ht...,neutral
1,1,Ariana Grande KIIS FM Yours Truly CD listening...,positive
2,2,Ariana Grande White House Easter Egg Roll in W...,positive
3,3,#CD #Musics Ariana Grande Sweet Like Candy 3.4...,positive
4,4,SIDE TO SIDE 😘 @arianagrande #sidetoside #aria...,neutral
...,...,...,...
12279,12279,@dansen17 update: Zac Efron kissing a puppy ht...,positive
12280,12280,#zac efron sex pic skins michelle sex https://...,neutral
12281,12281,First Look at Neighbors 2 with Zac Efron Shirt...,neutral
12282,12282,zac efron poses nude #lovely libra porn https:...,neutral


In [45]:
sem_eval_test.groupby('Polarity_Classification').size()

Polarity_Classification
negative    3972
neutral     5937
positive    2375
dtype: int64

In [181]:
#per sst vogliamo utilizzare il test set per sst-2


#facciamo binary classification
#il dataset va ristrutturato utilizzando le soglie 
from datasets import load_dataset
import pandas as pd
dataset = load_dataset("sst", "default")
sst=pd.DataFrame(list(zip(dataset['test']['sentence'],dataset['test']['label'])), columns=['Sentence','Polarity_Classification'])

classif=[]
for index,row in sst.iterrows():

        if float(row['Polarity_Classification']) <= 0.5:
                classif.append('negative')
        elif float(row['Polarity_Classification']) > 0.5:
                classif.append('positive')
       # else:
            #classif.append('neutral')
            

sst_test['Polarity_Classification']=classif
sst_test



#https://huggingface.co/datasets/sst/viewer/default/test

Reusing dataset sst (C:\Users\Fossati\.cache\huggingface\datasets\sst\default\1.0.0\b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff)


  0%|          | 0/3 [00:00<?, ?it/s]

,Sentence,Polarity_Classification
0,Effective but too-tepid biopic,positive
1,If you sometimes like to go to the movies to h...,positive
2,"Emerges as something rare , an issue movie tha...",positive
3,The film provides some great insight into the ...,positive
4,Offers that rare combination of entertainment ...,positive
...,...,...
2205,An imaginative comedy\/thriller .,positive
2206,"( A ) rare , beautiful film .",positive
2207,( An ) hilarious romantic comedy .,positive
2208,Never ( sinks ) into exploitation .,positive


In [182]:
from transformers import XLMRobertaTokenizerFast, XLMRobertaForSequenceClassification 
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
from transformers import Trainer, TrainingArguments
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
import torch
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix

In [183]:
def prediction_test_whelp(dataset, modello, tok, baseline):  #dataset fa riferimento al set di dati che vogliamo passare, modello dipende dal tipo di FT
#poi ci sarà anche la funzione per predire senza labels
    #modello = XLMRobertaForSequenceClassification.from_pretrained('xlm-roberta-base')
    #tok = XLMRobertaTokenizerFast.from_pretrained('xlm-roberta-base')
    test_texts=dataset['Sentence']
    test_labels=dataset['Polarity_Classification']

    if baseline == 'no':
        unique_labels={'neutral', 'positive', 'negative'}
        label2id={'negative': 2, 'neutral': 0, 'positive': 1}
        id2label={2:'negative', 0:'neutral', 1:'positive'}
        
    else:
        
        unique_labels={'positive', 'negative'}
        label2id={'negative': 1,  'positive': 0}
        id2label={1:'negative', 0:'positive'}
        

#train_encodings = tok(train_texts.tolist(), truncation=True, padding=True, max_length=max_length)
    test_encodings  = tok(test_texts.tolist(), truncation=True, padding=True, max_length=128)

#train_labels_encoded = [label2id[y] for y in train_labels.tolist()]
    test_labels_encoded  = [label2id[y] for y in test_labels.tolist()]
    

    class MyDataset(torch.utils.data.Dataset):
        def __init__(self, encodings, labels):
            self.encodings = encodings
            self.labels = labels

        def __getitem__(self, idx):
            item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
            item['labels'] = torch.tensor(self.labels[idx])
            return item

        def __len__(self):
            return len(self.labels)

#train_dataset = MyDataset(train_encodings, train_labels_encoded)
    test_dataset = MyDataset(test_encodings, test_labels_encoded)
    
    training_args = TrainingArguments( #sono sempre gli stessi alla fine
    num_train_epochs=4,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    learning_rate=2e-5,              # initial learning rate for Adam optimizer
    warmup_steps=100,                # number of warmup steps for learning rate scheduler (set lower because of small dataset size)
    weight_decay=0.01,               # strength of weight decay
    output_dir='./results',          # output directory
    logging_dir='./logs',            # directory for storing logs
    logging_steps=2500,               # number of steps to output logging (set lower because of small dataset size)
    evaluation_strategy='steps',     # evaluate during fine-tuning so that we can see progress
)
    
    trainer = Trainer(model=modello, args=training_args)  #basta avere il modello come parametro

    predicted_results=trainer.predict(test_dataset)
    
    if baseline == 'no':
        predicted_labels=[]
        count=0
        for el in predicted_results[0]:
    #la logica è cerchiamo argmax innogni lista, se viene zero, togliamo lo zero e prendiamo il seconod migliore
    #questo perchè abbiamo addestrato il modello su 3 classi fisse, per cui bisogna ricorrere ad un espediente per classificare su bench
            if int(np.argmax(predicted_results[0][count].tolist())) == 0:
                p=int(np.argmax(predicted_results[0][count].tolist()[1:]))
                if p==0:
                    p=1
                else:
                    p=2
            else:
                p=int(np.argmax(predicted_results[0][count].tolist()))
            predicted_labels.append(p)
            count+=1

    
    

    #predicted_labels = predicted_results.predictions.argmax(-1) # Get the highest probability prediction
 
    #predicted_labels = predicted_labels.flatten().tolist()      # Flatten the predictions into a 1D list
   
        predicted_labels = [id2label[l] for l in predicted_labels]  # Convert from integers back to strings for readability
    
    else:
        predicted_labels = predicted_results.predictions.argmax(-1) # Get the highest probability prediction
 
        predicted_labels = predicted_labels.flatten().tolist()      # Flatten the predictions into a 1D list
   
        predicted_labels = [id2label[l] for l in predicted_labels]  # Convert from integers back to strings for readability
        
    
    
    return print(classification_report(test_labels,predicted_labels)),print(confusion_matrix(test_labels,predicted_labels,labels=['positive','negative']))#aggiunta dopo
    #return predicted_results

In [184]:
def prediction_test(dataset, modello, tok):  #dataset fa riferimento al set di dati che vogliamo passare, modello dipende dal tipo di FT
#poi ci sarà anche la funzione per predire senza labels
    #modello = XLMRobertaForSequenceClassification.from_pretrained('xlm-roberta-base')
    #tok = XLMRobertaTokenizerFast.from_pretrained('xlm-roberta-base')
    test_texts=dataset['Sentence']
    test_labels=dataset['Polarity_Classification']

    unique_labels={'neutral', 'positive', 'negative'}
    label2id={'negative': 2, 'neutral': 0, 'positive': 1}
    id2label={2:'negative', 0:'neutral', 1:'positive'}
        
   
        

#train_encodings = tok(train_texts.tolist(), truncation=True, padding=True, max_length=max_length)
    test_encodings  = tok(test_texts.tolist(), truncation=True, padding=True, max_length=128)

#train_labels_encoded = [label2id[y] for y in train_labels.tolist()]
    test_labels_encoded  = [label2id[y] for y in test_labels.tolist()]
    

    class MyDataset(torch.utils.data.Dataset):
        def __init__(self, encodings, labels):
            self.encodings = encodings
            self.labels = labels

        def __getitem__(self, idx):
            item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
            item['labels'] = torch.tensor(self.labels[idx])
            return item

        def __len__(self):
            return len(self.labels)

#train_dataset = MyDataset(train_encodings, train_labels_encoded)
    test_dataset = MyDataset(test_encodings, test_labels_encoded)
    
    training_args = TrainingArguments( #sono sempre gli stessi alla fine
    num_train_epochs=4,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    learning_rate=2e-5,              # initial learning rate for Adam optimizer
    warmup_steps=100,                # number of warmup steps for learning rate scheduler (set lower because of small dataset size)
    weight_decay=0.01,               # strength of weight decay
    output_dir='./results',          # output directory
    logging_dir='./logs',            # directory for storing logs
    logging_steps=2500,               # number of steps to output logging (set lower because of small dataset size)
    evaluation_strategy='steps',     # evaluate during fine-tuning so that we can see progress
)
    
    trainer = Trainer(model=modello, args=training_args)  #basta avere il modello come parametro

    predicted_results=trainer.predict(test_dataset)
    

    predicted_labels = predicted_results.predictions.argmax(-1) # Get the highest probability prediction
 
    predicted_labels = predicted_labels.flatten().tolist()      # Flatten the predictions into a 1D list
   
    predicted_labels = [id2label[l] for l in predicted_labels]  # Convert from integers back to strings for readability
        
    
    
    return print(classification_report(test_labels,predicted_labels)),print(confusion_matrix(test_labels,predicted_labels,labels=['positive','negative']))#aggiunta dopo
    #return predicted_results

In [185]:
#m8

modello = XLMRobertaForSequenceClassification.from_pretrained('Polarity_base_xlm_ALSA')
tok = XLMRobertaTokenizerFast.from_pretrained('xlm-roberta-base')
#prediction_test(sem_eval_test, modello, tok)
prediction_test(sst_test, modello, tok)
print('\n')
prediction_test_whelp(sst_test, modello, tok, 'no')

loading configuration file Polarity_base_xlm_ALSA\config.json
Model config XLMRobertaConfig {
  "_name_or_path": "xlm-roberta-base",
  "architectures": [
    "XLMRobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.14.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_

C:\Users\Fossati\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Fossati\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Fossati\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

    negative       0.89      0.61      0.72      1143
     neutral       0.00      0.00      0.00         0
    positive       0.78      0.83      0.80      1067

    accuracy                           0.72      2210
   macro avg       0.55      0.48      0.51      2210
weighted avg       0.83      0.72      0.76      2210

[[888  90]
 [256 699]]




using `logging_steps` to initialize `eval_steps` to 2500
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
***** Running Prediction *****
  Num examples = 2210
  Batch size = 16


              precision    recall  f1-score   support

    negative       0.87      0.72      0.79      1143
    positive       0.75      0.88      0.81      1067

    accuracy                           0.80      2210
   macro avg       0.81      0.80      0.80      2210
weighted avg       0.81      0.80      0.80      2210

[[944 123]
 [322 821]]


(None, None)

In [186]:
#m7
#evaluation per io mgiliore modello prima della strategy finale, augmented 
modello = XLMRobertaForSequenceClassification.from_pretrained('Polarity_base_xlm_aug')
tok = XLMRobertaTokenizerFast.from_pretrained('xlm-roberta-base')
#prediction_test(sem_eval_test, modello, tok)
prediction_test(sst_test, modello, tok)
print('\n')
prediction_test_whelp(sst_test, modello, tok,'no')

loading configuration file Polarity_base_xlm_aug\config.json
Model config XLMRobertaConfig {
  "_name_or_path": "xlm-roberta-base",
  "architectures": [
    "XLMRobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.14.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_s

C:\Users\Fossati\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Fossati\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Fossati\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
using `logging_steps` to initialize `eval_steps` to 2500
PyTorch: setting

              precision    recall  f1-score   support

    negative       0.90      0.60      0.72      1143
     neutral       0.00      0.00      0.00         0
    positive       0.81      0.78      0.79      1067

    accuracy                           0.69      2210
   macro avg       0.57      0.46      0.50      2210
weighted avg       0.85      0.69      0.75      2210

[[834  78]
 [200 682]]




              precision    recall  f1-score   support

    negative       0.87      0.77      0.81      1143
    positive       0.78      0.87      0.82      1067

    accuracy                           0.82      2210
   macro avg       0.82      0.82      0.82      2210
weighted avg       0.82      0.82      0.82      2210

[[931 136]
 [268 875]]


(None, None)

In [ ]:
#EVALUATION STRATEGIA FINALE

In [187]:
#mBERT


modello = DistilBertForSequenceClassification.from_pretrained('Polarity_fin_mbert')
tok = DistilBertTokenizerFast.from_pretrained('distilbert-base-multilingual-cased')
#prediction_test(sem_eval_test, modello, tok)
prediction_test(sst_test, modello, tok)
print('\n')
prediction_test_whelp(sst_test, modello, tok,'no')



loading configuration file Polarity_fin_mbert\config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-multilingual-cased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.14.1",
  "vocab_size": 119547
}

loading weights file Polarity_fin_mbert\pytorch_model.bin
All model checkpoint weights were used when initi

C:\Users\Fossati\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Fossati\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Fossati\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
using `logging_steps` to initialize `eval_steps` to 2500
PyTorch: setting

              precision    recall  f1-score   support

    negative       0.75      0.51      0.61      1143
     neutral       0.00      0.00      0.00         0
    positive       0.72      0.62      0.67      1067

    accuracy                           0.56      2210
   macro avg       0.49      0.38      0.42      2210
weighted avg       0.74      0.56      0.64      2210

[[660 190]
 [253 582]]




              precision    recall  f1-score   support

    negative       0.72      0.73      0.73      1143
    positive       0.71      0.70      0.70      1067

    accuracy                           0.72      2210
   macro avg       0.72      0.72      0.72      2210
weighted avg       0.72      0.72      0.72      2210

[[751 316]
 [313 830]]


(None, None)

In [ ]:
#tra gli mBERT è il migliore quello finale
#tuttavia risultati non comparabili ad mbert

In [188]:
#xlmr

modello = XLMRobertaForSequenceClassification.from_pretrained('Polarity_fin_xlmr')
tok = XLMRobertaTokenizerFast.from_pretrained('xlm-roberta-base')
#prediction_test(sem_eval_test, modello, tok)
prediction_test(sst_test, modello, tok)
print('\n')
prediction_test_whelp(sst_test, modello, tok,'no')


loading configuration file Polarity_fin_xlmr\config.json
Model config XLMRobertaConfig {
  "_name_or_path": "xlm-roberta-base",
  "architectures": [
    "XLMRobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.14.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size"

C:\Users\Fossati\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Fossati\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Fossati\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
using `logging_steps` to initialize `eval_steps` to 2500
PyTorch: setting

              precision    recall  f1-score   support

    negative       0.88      0.59      0.71      1143
     neutral       0.00      0.00      0.00         0
    positive       0.86      0.69      0.76      1067

    accuracy                           0.64      2210
   macro avg       0.58      0.43      0.49      2210
weighted avg       0.87      0.64      0.73      2210

[[734  88]
 [122 675]]




              precision    recall  f1-score   support

    negative       0.86      0.78      0.82      1143
    positive       0.78      0.86      0.82      1067

    accuracy                           0.82      2210
   macro avg       0.82      0.82      0.82      2210
weighted avg       0.82      0.82      0.82      2210

[[920 147]
 [254 889]]


(None, None)

In [ ]:
#computiamo infine le baseline sui test

In [189]:
#mBERT baseline


modello = DistilBertForSequenceClassification.from_pretrained('distilbert-base-multilingual-cased',num_labels=3)
tok = DistilBertTokenizerFast.from_pretrained('distilbert-base-multilingual-cased')
#prediction_test(sem_eval_test, modello, tok)
prediction_test(sst_test, modello, tok)
print('\n')
modello = DistilBertForSequenceClassification.from_pretrained('distilbert-base-multilingual-cased',num_labels=2)
prediction_test_whelp(sst_test, modello, tok, 'yes')


loading configuration file https://huggingface.co/distilbert-base-multilingual-cased/resolve/main/config.json from cache at C:\Users\Fossati/.cache\huggingface\transformers\cf37a9dc282a679f121734d06f003625d14cfdaf55c14358c4c0b8e7e2b89ac9.7a727bd85e40715bec919a39cdd6f0aba27a8cd488f2d4e0f512448dcd02bf0f
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.14.1",
  "vocab_size": 119547
}

loading we

C:\Users\Fossati\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Fossati\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Fossati\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

    negative       0.52      1.00      0.68      1143
    positive       0.00      0.00      0.00      1067

    accuracy                           0.52      2210
   macro avg       0.26      0.50      0.34      2210
weighted avg       0.27      0.52      0.35      2210

[[   0 1067]
 [   0 1143]]




loading configuration file https://huggingface.co/distilbert-base-multilingual-cased/resolve/main/config.json from cache at C:\Users\Fossati/.cache\huggingface\transformers\cf37a9dc282a679f121734d06f003625d14cfdaf55c14358c4c0b8e7e2b89ac9.7a727bd85e40715bec919a39cdd6f0aba27a8cd488f2d4e0f512448dcd02bf0f
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.14.1",
  "vocab_size": 119547
}

loading weights file https://huggingface.co/distilbert-base-multilingual-cased/resolve/main/pytorch_model.bin from cache at C:\Users\Fossati/.cache\huggingface\tran

              precision    recall  f1-score   support

    negative       0.56      0.44      0.49      1143
    positive       0.51      0.63      0.57      1067

    accuracy                           0.53      2210
   macro avg       0.54      0.53      0.53      2210
weighted avg       0.54      0.53      0.53      2210

[[674 393]
 [644 499]]


(None, None)

In [190]:
#xlmr baseline

modello = XLMRobertaForSequenceClassification.from_pretrained('xlm-roberta-base',num_labels=3)
tok = XLMRobertaTokenizerFast.from_pretrained('xlm-roberta-base')
prediction_test(sst_test, modello, tok)
print('\n')
modello = XLMRobertaForSequenceClassification.from_pretrained('xlm-roberta-base',num_labels=2)
prediction_test_whelp(sst_test, modello, tok, 'yes')


loading configuration file https://huggingface.co/xlm-roberta-base/resolve/main/config.json from cache at C:\Users\Fossati/.cache\huggingface\transformers\87683eb92ea383b0475fecf99970e950a03c9ff5e51648d6eee56fb754612465.dfaaaedc7c1c475302398f09706cbb21e23951b73c6e2b3162c1c8a99bb3b62a
Model config XLMRobertaConfig {
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute

C:\Users\Fossati\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Fossati\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Fossati\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

    negative       0.52      1.00      0.68      1143
    positive       0.00      0.00      0.00      1067

    accuracy                           0.52      2210
   macro avg       0.26      0.50      0.34      2210
weighted avg       0.27      0.52      0.35      2210

[[   0 1067]
 [   0 1143]]




loading configuration file https://huggingface.co/xlm-roberta-base/resolve/main/config.json from cache at C:\Users\Fossati/.cache\huggingface\transformers\87683eb92ea383b0475fecf99970e950a03c9ff5e51648d6eee56fb754612465.dfaaaedc7c1c475302398f09706cbb21e23951b73c6e2b3162c1c8a99bb3b62a
Model config XLMRobertaConfig {
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.14.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 250002
}

loading weights file https://huggingface.co/

              precision    recall  f1-score   support

    negative       0.00      0.00      0.00      1143
    positive       0.48      1.00      0.65      1067

    accuracy                           0.48      2210
   macro avg       0.24      0.50      0.33      2210
weighted avg       0.23      0.48      0.31      2210

[[1067    0]
 [1143    0]]


C:\Users\Fossati\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Fossati\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Fossati\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(None, None)

In [ ]:
#per il modello migliore si vuole poi computare per ogni lingua e per il benchmark la confusion matrix, in maniera tale da capire
#dove ci sono gli errori per ogni lingua

In [ ]:
#sul benchmark binary classification serve a poco la confusione matrix

In [ ]:
#il modello migliore ottiene 82% di accuracy sul dataset di benchmark per binary class